In [3]:
! pip install datasets


[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [1]:
! pip install transformers datasets tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.8 MB/s eta 0:00:00
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.9 MB/s eta 0:00:00
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-1-py2.py3-none-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-4.25.5-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorboard-2.17.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.6.0-py3-n

  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached werkzeug-3.0.4-py3-none-any.whl.metadata (3.7 kB)
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 5.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 4.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.1/236.1 MB 5.1 MB/s eta 0:00:0000:0100:02
Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.6 MB/s eta 0:00:00a 0:00:01
Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
Using cached fsspec-2024.6.1-py3-no

In [8]:
from datasets import load_dataset

# 加载IMDb数据集
dataset = load_dataset('imdb')

# 选取少量数据进行实验
train_dataset = dataset['train'].shuffle(seed=42).select(range(5000))
test_dataset = dataset['test'].shuffle(seed=42).select(range(1000))

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [9]:
from transformers import AutoTokenizer, TFBertForSequenceClassification

# 加载BERT的分词器和模型
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

2024-10-20 10:41:45.387044: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-10-20 10:41:45.387102: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2024-10-20 10:41:45.387113: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2024-10-20 10:41:45.387158: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-20 10:41:45.387180: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initiali

In [10]:
def preprocess_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

# 对训练集和测试集进行分词和编码
train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
import tensorflow as tf

# 转换为TensorFlow格式的dataset
train_dataset.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'label'])

# 从dataset中提取Tensor
train_features = {x: tf.convert_to_tensor(train_dataset[x]) for x in ['input_ids', 'attention_mask']}
train_labels = tf.convert_to_tensor(train_dataset['label'])
test_features = {x: tf.convert_to_tensor(test_dataset[x]) for x in ['input_ids', 'attention_mask']}
test_labels = tf.convert_to_tensor(test_dataset['label'])

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels)).shuffle(1000).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((test_features, test_labels)).batch(16)

In [13]:
# 编译模型
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# 训练模型
history = model.fit(train_dataset, validation_data=test_dataset, epochs=3)

Epoch 1/3


2024-10-20 10:41:55.644112: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-10-20 10:41:56.534879: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:966] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp_10.


313/313 [==============================] - 165s 460ms/step - loss: 0.4462 - accuracy: 0.7902 - val_loss: 0.3380 - val_accuracy: 0.8590
Epoch 2/3
313/313 [==============================] - 133s 423ms/step - loss: 0.2757 - accuracy: 0.8888 - val_loss: 0.3443 - val_accuracy: 0.8580
Epoch 3/3
313/313 [==============================] - 129s 414ms/step - loss: 0.1769 - accuracy: 0.9348 - val_loss: 0.3892 - val_accuracy: 0.8630


In [14]:
# 在测试集上评估模型
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

63/63 [==============================] - 10s 154ms/step - loss: 0.3892 - accuracy: 0.8630
Test Loss: 0.3891863226890564
Test Accuracy: 0.8629999756813049
